**Importação de bibliotecas e base de treino**

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
train = pd.read_csv("../input/train.csv")

**Análise de dados e seleção de parâmetros**

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.dtypes.value_counts()

Iremos utilizar somente as colunas cujos valores são numéricos

In [ ]:
train.select_dtypes(include='object').columns

In [ ]:
train2 = train.drop(columns = ['Id', 'idhogar', 'dependency', 'edjefe', 'edjefa'])

Verificando colunas com dados faltantes

In [ ]:
null_columns = train2.columns[train2.isnull().any()]
train2[null_columns].isnull().sum()

Como as colunas v2a1, v18q1, rez_esc possuem muitos dados faltantes, elas serão removidas. E as linhas com outros dados faltantes serão removidas.

In [ ]:
train3 = train2.drop(columns = ['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned'])
ntrain = train3.dropna()

In [ ]:
ntrain.head()

**Adaptando a base de testes**

In [ ]:
test = pd.read_csv("../input/test.csv")
null_columns = test.columns[test.isnull().any()]
test[null_columns].isnull().sum()

In [ ]:
Xtest = test.drop(columns = ['Id', 'idhogar', 'dependency', 'edjefe', 'edjefa', 'v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned'])
Xntest = Xtest.dropna()
Xid = test.drop(columns = ['idhogar', 'dependency', 'edjefe', 'edjefa', 'v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned'])
Xnid = Xid.dropna()

Xid.shape


**Criando modelo kNN**

In [ ]:
Xtrain = ntrain.drop(columns = ["Target"])
Ytrain = ntrain.Target

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
scores = cross_val_score(knn, Xtrain, Ytrain, cv=10)

In [ ]:
np.mean(scores)

Determinando melhor K

maiork = 1
maiormedia = 0

for k in range(1,100):
    knn = KNeighborsClassifier(n_neighbors = k)
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=10)
    if np.mean(scores) > maiormedia:
        maiormedia = np.mean(scores)
        maiork = k

maiork

**Executando modelo (melhor k encontrado = 97)**

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 97)

In [ ]:
knn.fit(Xtrain, Ytrain)

In [ ]:
Ypred = knn.predict(Xntest)

In [ ]:
result = np.vstack((Xid["Id"], Ypred)).T
x = ["Id","Target"]
Resultado = pd.DataFrame(columns = x, data = result)
Resultado.to_csv("results.csv", index = False)
Resultado